In [2]:
# imports
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
import numpy as np
import sys, os, os.path
import subprocess
import time
import random
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import skimage.io as sio
import skimage.transform as sm

from models import *


In [21]:
# load model
network='inception_net'
device='cpu'
# device='cuda:0'
net = eval(network).Net(3).to(device)
# initialize randomly
for p in net.parameters():
    nn.init.normal_(p,0, 0.1)

In [22]:
# list trainable variables + GPU size + feature layers + decision layers
params=0
for p in net.parameters():
#     print p.shape
    p_values=1
    for d in list(p.shape[:]):
        p_values*=d
    params+=p_values
    
feature_layers, decision_layers=0,0
# for l in net.network.network:
#     if 'Conv' in str(l) or 'Linear' in str(l): 
#         feature_layers+=1
# for l in net.network.fc:
#     if 'Conv' in str(l) or 'Linear' in str(l): 
#         decision_layers+=1
print("feature layers: {0}, decision layers: {1}, parameters: {2}".format(feature_layers, 
                                                                          decision_layers, 
                                                                          params))
print("\n")
nvidia=subprocess.check_output(["nvidia-smi"])
print nvidia

# print net

feature layers: 0, decision layers: 0, parameters: 24351718


Thu Mar 14 16:47:19 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.43       Driver Version: 418.43       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN X (Pascal)    Off  | 00000000:01:00.0  On |                  N/A |
| 46%   68C    P2    65W / 250W |   1679MiB / 12187MiB |      1%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:              

In [5]:
# load one run of esatv3
run_dir='/esat/opal/kkelchte/docker_home/pilot_data/esatv3_expert/00000_esatv3/RGB'
images=[run_dir+'/'+im for im in sorted(os.listdir(run_dir)) if im.endswith('jpg')]
data=[]
im_size=net.default_image_size
for img_file in images:
    img = sio.imread(img_file)    
    img = np.swapaxes(img,1,2)
    img = np.swapaxes(img,0,1)
    scale_height = int(np.floor(img.shape[1]/im_size[1]))
    scale_width = int(np.floor(img.shape[2]/im_size[2]))
    img = img[:,::scale_height,::scale_width]
    img = sm.resize(img,im_size,mode='constant').astype(np.float16)
    img -= 0.5
    assert len(img) != 0, '[data] Loading image failed: {}'.format(img_file)
    data.append(img)
data=np.asarray(data)

/users/visics/kkelchte/tensorflow_1.8/lib/python2.7/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [23]:
# evaluate model (without autograd) and measure FPS
load_time=[]
predict_time=[]
for i in range(data.shape[0]):
    if i%100 == 0: print("Frame {0}/{1}".format(i,data.shape[0]))
    stime=time.time()
    inputs=torch.from_numpy(np.expand_dims(data[0],0)).type(torch.FloatTensor).to(device)
    load_time.append(time.time()-stime)
    stime=time.time()
    predictions = net.forward(inputs, train=False)
    predict_time.append(time.time()-stime)
print("load: {0}({1}), prediction: {2}({3})".format(np.mean(load_time),
                                                                          np.std(load_time),
                                                                          np.mean(predict_time),
                                                                          np.std(predict_time)))
print("speed: {0}".format(1/np.mean(predict_time)))

torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size(

torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224

torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224

torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224

torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224

...

torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224

torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224

torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224

torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224

torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3])
torch.Size([1, 3, 224